In [1]:
import numpy as np, pandas as pd, warnings, os, openai, json
from tqdm.auto import tqdm
from openai import OpenAI
warnings.filterwarnings('ignore')
# import requests
from typing import List, Tuple, Union
from langchain_ollama.embeddings import OllamaEmbeddings
# import pickle
from sklearn.metrics.pairwise import cosine_similarity
# from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda
from langchain.memory import ConversationBufferMemory
# from langchain_ollama.chat_models import 
# from langchain.schema import Sys
from langgraph.graph import StateGraph, END

/home/ubuntu2204/miniconda3/envs/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
key = open('../../../../api_key.txt','r')
api_key = key.read()
openai.api_key = api_key

base_ = open('../../../../base_url.txt','r')
base_url = base_.read()
# openai.api_key = api_key

# GPT.ver

In [3]:
def invoke(
           prompt,
           api_key=api_key,
           model="gpt-4.1",
           temperature=0):
    client = OpenAI(api_key=api_key)

    # 메시지 구성
    messages = [
        # {"role": "system", "content": prompt},
        {"role": "user", "content": prompt}
    ]
    # 스트리밍 요청
    stream = client.chat.completions.create(
        model=model,
        # messages=messages,
        messages=prompt,
        temperature=temperature,
        # stream=True  # ⭐ 핵심 옵션!
    )

    # generator 반환 (chunk 단위 텍스트 출력)
    return stream

memory = ConversationBufferMemory(return_messages=True)

/tmp/ipykernel_1106/450895685.py:25: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


# Fucntion

In [4]:
class OllamaSentenceTransformer():
    def __init__(
            self,
            *args,
            **kargs,
            ) -> None:
                # self.base_url = kargs.get("base_url", "http://localhost:11434")
                self.model = kargs.get("model","bge-m3")
                self.embedding_model = OllamaEmbeddings(
                            model = self.model,
                            base_url = base_url,
                        )
                        
    
    def encode(
            self,
            documents:Union[str, List[str], np.ndarray],
            *args,
            **kargs,
        )-> np.ndarray:
        if isinstance(documents, str):
            document_embeddings = self.embedding_model.embed_query(documents)
            return np.array(document_embeddings)
        
        if isinstance(documents, np.ndarray):
            documents = documents.tolist()
        
        document_embeddings = [self.embedding_model.embed_query(s) for s in documents]
        return np.array(document_embeddings)
        


In [5]:
# Node 1: 질문 내용 구분 함수
def classify_question(stats):
    # input_question = {"question": "3 곱하기 7은 얼마야?"}

    question = stats["question"]
    messages = [
                    {"role": "system", 
                        "content": f"""당신은 의료분야의 전문 지식을 갖춘 병원 안내데스크 직원입니다.
                        현재 환자가 들어와 간단한 질의에 대하여 상담을 받고 있습니다. 저희 병원에서 받고있는 과는 '산부인과','소아청소년과','응급의학과','내과'입니다.
                        해당 과에 대한 질문이 들어온다면 해당 과를 뱉어주세요.
                        만약 제시한 과 중 어느 곳에도 속하지 않는다면 'General'을 반환해줘.
                        산부인과 관련 질문이 들어오면 '산부인과', 소아청소년과 관련 질문이 들어오면 '소아청소년과', 응급의학과 관련 질문이 들어오면 '응급의학과', 내과 관련 질문이 들어오면 '내과' 
                        너가 답변할 수 있는 내용은 ['산부인과','내과','응급의학과','소아청소년과','General'] 뿐이야.
                        """
                        },
                    {"role": "user", 
                    "content": question}
                ]
    response = invoke(messages).choices[0].message.content
    stats['type']=response
    return stats

In [7]:
question = '배속이 너무 아프고 땀이 너무 나.'
messages = [
                    {"role": "system", 
                        "content": f"""당신은 의료분야의 전문 지식을 갖춘 병원 안내데스크 직원입니다.
                        현재 환자가 들어와 간단한 질의에 대하여 상담을 받고 있습니다. 저희 병원에서 받고있는 과는 '산부인과','소아청소년과','응급의학과','내과'입니다.
                        해당 과에 대한 질문이 들어온다면 해당 과를 뱉어주세요.
                        만약 제시한 과 중 어느 곳에도 속하지 않는다면 'General'을 반환해줘.
                        산부인과 관련 질문이 들어오면 '산부인과', 소아청소년과 관련 질문이 들어오면 '소아청소년과', 응급의학과 관련 질문이 들어오면 '응급의학과', 내과 관련 질문이 들어오면 '내과' 
                        너가 답변할 수 있는 내용은 '산부인과','내과','응급의학과','소아청소년과','General'] 뿐이야.
                        """
                        },
                    {"role": "user", 
                    "content": question}
                ]

In [8]:
response = invoke(messages)

In [16]:
def generate_response(state, category):
    history = memory.chat_memory.messages  # 이전 대화
    current_question = state["question"]
    
    # 현재 질문과 카테고리에 맞는 프롬프트 구성
    prompt = f"사용자가 이렇게 질문했습니다: '{current_question}'\n"
    # if category == "Normal":
    #     prompt += "→ 질문에 대해서 개념 및 풀이과정을 상세히 설명해주세요."
    if category == "산부인과":
        prompt += "→ 산부인과 질의입니다. 당신은 산부인과의 전설입니다. 물음에 대하여 친철하고, 상세히 설명해주세요."
    elif category == "소아청소년과":
        prompt += "→ 소아청소년과 질의입니다. 당신은 소아청소년과의 전설입니다. 물음에 대하여 친철하고, 상세히 설명해주세요."
    elif category == "응급의학과":
        prompt += "→ 응급의학과 질의입니다. 당신은 응급의학과의 전설입니다. 물음에 대하여 친철하고, 상세히 설명해주세요."
    elif category == "내과":
        prompt += "→ 내과 질의입니다. 당신은 내과의 전설입니다. 물음에 대하여 친철하고, 상세히 설명해주세요."
    else:
        prompt += "→ 사용자의 물음에 친절히 답해주세요."
    
    # 모델 호출
    answer = invoke(history + [{"role": "user", "content": prompt}])
    response = answer.choices[0].message.content
    
    # 응답을 메모리에 저장
    memory.chat_memory.add_user_message(current_question)
    memory.chat_memory.add_ai_message(response)
    
    state["response"] = response
    return state

def decide_next(state):
    return state["type"]

In [17]:
# Node Wrappers
baby_node = RunnableLambda(lambda s: generate_response(s, "산부인과"))
kid_node = RunnableLambda(lambda s: generate_response(s, "소아청소년과"))
emergency_node = RunnableLambda(lambda s: generate_response(s, "응급의학과"))
internal_node = RunnableLambda(lambda s: generate_response(s, "내과"))
general_node = RunnableLambda(lambda s: generate_response(s, "General"))

In [18]:
# ✅ LangGraph 구성
state_type = dict
graph = StateGraph(state_schema=state_type)

graph.add_node("classify", RunnableLambda(classify_question))
graph.add_node("산부인과", baby_node)
graph.add_node("소아청소년과", kid_node)
graph.add_node("응급의학과", emergency_node)
graph.add_node("내과", internal_node)
graph.add_node("General", general_node)


graph.add_edge("산부인과", END)
graph.add_edge("소아청소년과", END)
graph.add_edge("응급의학과", END)
graph.add_edge("내과", END)
graph.add_edge("General", END)


In [19]:
graph.set_entry_point("classify")

graph.add_conditional_edges(
    "classify",
    path=RunnableLambda(decide_next),
    path_map={
        "산부인과": "산부인과",
        "소아청소년과": "소아청소년과",
        "응급의학과": "응급의학과",
        "내과": "내과",
        "General": "General",
    
    }
)

In [20]:
chat_agent = graph.compile()


In [21]:
chat_agent.invoke({"question": '배속이 너무 아프고 땀이 너무 나.'})


{'question': '배속이 너무 아프고 땀이 너무 나.',
 'type': '내과',
 'response': '안녕하세요. 말씀해주신 "배가 너무 아프고 땀이 너무 난다"는 증상은 여러 가지 원인에 의해 발생할 수 있습니다. 내과적으로 중요한 증상일 수 있으니, 아래 내용을 참고해주시고 필요시 빠르게 진료를 받으시길 권유드립니다.\n\n### 1. 증상의 원인\n배가 아프고 땀이 나는 증상은 다음과 같은 다양한 원인에 의해 발생할 수 있습니다.\n\n- **급성 복통(급성 복부질환)**: 맹장염(충수염), 담낭염, 췌장염, 장폐색, 위궤양 천공 등\n- **소화기계 감염**: 식중독, 바이러스성 장염 등\n- **심혈관계 문제**: 심근경색(특히 상복부 통증으로 나타날 수 있음)\n- **내분비 문제**: 저혈당, 갑상선 기능 항진증 등\n- **기타**: 신장결석, 요로감염, 산통 등\n\n### 2. 위험 신호(경고 증상)\n아래와 같은 증상이 동반된다면 즉시 응급실 방문이 필요합니다.\n\n- 통증이 갑자기 심해지거나, 참을 수 없을 정도로 심함\n- 식은땀, 창백함, 어지러움, 의식 저하\n- 구토, 혈변, 흑색변, 혈뇨\n- 호흡곤란, 가슴통증 동반\n- 고열(38도 이상) 또는 오한\n\n### 3. 현재 할 수 있는 조치\n- **금식**: 원인 모를 복통이 있을 때는 음식이나 물 섭취를 일시적으로 중단하는 것이 좋습니다.\n- **안정**: 무리한 움직임을 피하고, 편안한 자세로 휴식하세요.\n- **체온 측정**: 열이 있는지 확인하세요.\n- **증상 기록**: 통증의 위치, 양상, 지속 시간, 동반 증상(구토, 설사, 소변 이상 등)을 기록해두면 진료에 도움이 됩니다.\n\n### 4. 병원 방문이 필요한 경우\n- 통증이 1~2시간 이상 지속되거나 점점 심해질 때\n- 위의 위험 신호가 있을 때\n- 이전에 경험하지 못한 새로운 통증일 때\n- 만성질환(당뇨, 심장질환 등)이 있는 경우\n\n### 5. 진료 시 예상 검사\n- 혈액

In [14]:
prompt = f"""
                너는 사용자의 질문에 똑똑하고 정확하게 대답해주는 도우미야.
                포카라에 대해서 설명해줘
                """

In [15]:
response = invoke(prompt).choices[0].message.content

In [17]:
print(response)

네, 포카라(Pokhara)에 대해 설명해드릴게요!

**포카라**는 네팔 중서부에 위치한 도시로, 카트만두에 이어 네팔에서 두 번째로 큰 도시입니다. 해발 약 800m에 자리잡고 있으며, 히말라야 산맥의 아름다운 풍경과 자연 경관으로 매우 유명합니다.

### 주요 특징

1. **관광 명소**
   - **페와 호수(Phewa Lake)**: 포카라에서 가장 유명한 호수로, 배를 타고 호수 한가운데에 있는 바라히 사원(Barahi Temple)까지 갈 수 있습니다.
   - **사랑코트(Sarangkot)**: 일출과 히말라야 산맥(특히 마차푸차레와 안나푸르나 봉우리)을 감상할 수 있는 전망대로 유명합니다.
   - **데이비스 폭포(Davis Falls)**: 강물이 지하로 빨려 들어가는 독특한 폭포입니다.
   - **세계 평화 파고다(World Peace Pagoda)**: 멋진 전망과 평화로운 분위기를 즐길 수 있는 불교 사원입니다.

2. **등산과 트레킹의 관문**
   - 포카라는 안나푸르나(Annapurna) 트레킹의 출발점으로, 세계 각지의 트레커들이 모이는 곳입니다.
   - 패러글라이딩, 래프팅, 산악자전거 등 다양한 액티비티도 인기가 많아요.

3. **자연 경관**
   - 히말라야의 눈 덮인 봉우리와 푸른 호수, 울창한 숲이 어우러져 그림 같은 풍경을 자랑합니다.

4. **기후**
   - 포카라는 카트만두보다 온화하고, 연중 대부분 날씨가 좋습니다.

### 요약
포카라는 히말라야 산맥의 아름다운 풍경, 트레킹과 다양한 액티비티, 그리고 평화로운 분위기로 유명한 네팔의 대표적인 관광 도시입니다. 자연을 좋아하거나, 트레킹과 액티비티를 즐기고 싶은 분들에게 꼭 추천하는 여행지입니다!
